In [1]:
import tensorflow as tf
from keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
   1/1875 [..............................] - ETA: 5:39 - loss: 2.2823 - accuracy: 0.0938

2023-06-24 17:32:44.592826: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 9s 5ms/step - loss: 0.2083 - accuracy: 0.9410 - val_loss: 0.0861 - val_accuracy: 0.9753
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0755 - accuracy: 0.9777 - val_loss: 0.0618 - val_accuracy: 0.9817
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0570 - accuracy: 0.9832 - val_loss: 0.0580 - val_accuracy: 0.9811
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0470 - accuracy: 0.9857 - val_loss: 0.0572 - val_accuracy: 0.9823
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0569 - accuracy: 0.9821
Accuracy before quantization: 0.9821000099182129


INFO:tensorflow:Assets written to: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmp362zaaf7/assets


INFO:tensorflow:Assets written to: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmp362zaaf7/assets
2023-06-24 17:33:34.087352: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-24 17:33:34.087365: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-24 17:33:34.087724: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmp362zaaf7
2023-06-24 17:33:34.088426: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-24 17:33:34.088431: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmp362zaaf7
2023-06-24 17:33:34.090446: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-06-24 17:33:34.090860: I tensorflow/cc/saved_model/load

ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_reshape_input:0 

In [5]:
import numpy as np

In [10]:
# Evaluate the model accuracy before quantization
_, accuracy = model.evaluate(x_test, y_test)
print("Accuracy before quantization:", accuracy)

# Convert the model to a quantized version
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized model to a file
with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

# Convert x_test to FLOAT32
x_test = x_test.astype('float32')

# Select a single image from x_test for evaluation
image_index = 0  # Choose the index of the image to evaluate
image = x_test[image_index]
image = image.reshape(28, 28, 1)  # Reshape to match the input shape of the quantized model

# Load the quantized model
interpreter = tf.lite.Interpreter(model_path='quantized_model.tflite')
interpreter.allocate_tensors()

# Get input and output details of the quantized model
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Check the expected input shape of the quantized model
input_shape = input_details[0]['shape']
print("Expected input shape:", input_shape)

# Resize the input image to match the expected input shape
image_resized = tf.image.resize(image, (input_shape[1], input_shape[2]))
image_input = image_resized.numpy().reshape(input_shape)

# Set the input tensor to the quantized model
interpreter.set_tensor(input_details[0]['index'], image_input)
interpreter.invoke()

# Get the output tensor of the quantized model
output_tensor = interpreter.get_tensor(output_details[0]['index'])

# Perform prediction on the quantized model
predicted_label = np.argmax(output_tensor)
true_label = y_test[image_index]
print("Predicted label:", predicted_label)
print("True label:", true_label)

313/313 [==============================] - 1s 2ms/step - loss: 0.0569 - accuracy: 0.9821
Accuracy before quantization: 0.9821000099182129


INFO:tensorflow:Assets written to: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpz5_50vr4/assets


INFO:tensorflow:Assets written to: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpz5_50vr4/assets


Expected input shape: [ 1 28 28]
Predicted label: 7
True label: 7


2023-06-24 17:42:33.198116: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-24 17:42:33.198131: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-24 17:42:33.198205: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpz5_50vr4
2023-06-24 17:42:33.198932: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-24 17:42:33.198978: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpz5_50vr4
2023-06-24 17:42:33.200814: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-06-24 17:42:33.217545: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpz5_50vr4
2023-06-

In [14]:
# evaluate quantized model
# Convert y_test to int64
y_test = y_test.astype('int64')

# Initialize a variable to store the number of correct predictions
correct_predictions = 0

# Iterate over the entire test set
for i in range(len(x_test)):
    # Prepare the input image
    image = x_test[i].reshape(28, 28, 1)  # Reshape to match the input shape of the quantized model
    image = tf.image.resize(image, (input_shape[1], input_shape[2]))  # Resize the image
    image_input = image.numpy().reshape(input_shape)  # Reshape the image to match the input shape

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], image_input)

    # Invoke the interpreter
    interpreter.invoke()

    # Get the output tensor
    output_tensor = interpreter.get_tensor(output_details[0]['index'])

    # Perform prediction
    predicted_label = np.argmax(output_tensor)
    true_label = y_test[i]

    # Compare the predicted label to the true label
    if predicted_label == true_label:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / len(x_test)

print("Accuracy after quantization:", accuracy)



Accuracy after quantization: 0.982


# Conclusion

## prequantization accuracy:
313/313 [==============================] - 1s 2ms/step - loss: 0.0569 - accuracy: 0.9821

## postquantization accuracy:
Accuracy after quantization: 0.982


In [16]:
def representative_data_gen():
    for input_value in x_test:
        yield [np.array(input_value, dtype=np.float32, ndmin=2)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8 for TensorFlow Lite < 2.4.0
converter.inference_output_type = tf.int8  # or tf.uint8 for TensorFlow Lite < 2.4.0
quantized_tflite_model = converter.convert()



INFO:tensorflow:Assets written to: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpbn8sgdmw/assets


INFO:tensorflow:Assets written to: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpbn8sgdmw/assets
/opt/homebrew/Caskroom/miniconda/base/envs/LanguageDetection/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-06-24 17:50:52.779272: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-24 17:50:52.779286: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-24 17:50:52.779362: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/f3/jxhschwd67n3mthj9bst1d_w0000gn/T/tmpbn8sgdmw
2023-06-24 17:50:52.780134: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-24 17:50:52.780145: I tensorflow/cc/saved_model/reader.cc:130] Re

RuntimeError: tensorflow/lite/kernels/reshape.cc:85 num_input_elements != num_output_elements (784 != 21952)Node number 3 (RESHAPE) failed to invoke.